In [1]:
import numpy as np
import subprocess
from PIL import Image
import pyqrcode
from pyzbar.pyzbar import decode
import os

In [2]:
def get_qr_array(m):
    qr = pyqrcode.create(m, error='L', version=40, mode='binary')
    t = np.array([[1-int(z) for z in x] for x in qr.text().split('\n')[:-1]])
    return t

In [3]:
def qr_rgb_1_encode(frame, size=(400, 400)):
    m = ''.join(str(x) for x in np.random.randint(0, 2, (2000,)).tolist())
    t = get_qr_array(m)
    rgb = np.repeat(t[:, :, np.newaxis], 3, axis=2)
    rgb = np.uint8(rgb * 255)
    img = Image.fromarray(rgb, 'RGB')
    img = img.resize(size)
    frame.paste(img)
    return frame, m

In [4]:
def qr_rgb_3_encode(frame, size=(400, 400)):
    ms = ''
    rgb = []
    for i in range(3):
        m = ''.join(str(x) for x in np.random.randint(0, 2, (100,)).tolist())
        ms += m
        rgb.append(get_qr_array(m))
    rgb = np.stack(rgb, axis=2)
    rgb = np.uint8(rgb * 255)
    img = Image.fromarray(rgb, 'RGB')
    img = img.resize(size)
    frame.paste(img)
    return frame, ms

In [5]:
def qr_rgb_1_decode(frame, size=(400, 400)):
    f = frame.crop((0, 0, size[0], size[1]))
    res = decode(f)
    if not res:
        return ''
    return decode(f)[0].data.decode('utf-8')
   

In [6]:
def qr_rgb_3_decode(frame, size=(400, 400)):
    f = np.array(frame.crop((0, 0, size[0], size[1])))
    ms = ''
    for i in range(3):
        t = f[:, :, i][:, :, np.newaxis]
        t = np.repeat(t, 3, axis=2)
        t = Image.fromarray(t, 'RGB')
        if res:
            ms += decode(t)[0].data.decode('utf-8')
    return ms

In [7]:
def put_data(directory):
    '''embedes all of the png files in a directory with a qr code and saves them in place.
    returns the string representations of the qr codes in a list'''
    m = len(os.listdir(directory)) * [None]
    for i, img in enumerate(os.listdir(directory)):
        frame = Image.open(directory + '/' + img)
        f, m[i] = qr_rgb_1_encode(frame)
        f.save(directory + '/' + img)
    return m

In [8]:
def get_data(directory):
    '''reads the qr codes in all of the png files in the directory
    returns the string representations of the qr codes in a list'''
    m = len(os.listdir(directory)) * [None]
    for i, img in enumerate(os.listdir(directory)):
        frame = Image.open(directory + '/' + img)
        m[i] = qr_rgb_1_decode(frame)
    return m
    
    

In [ ]:
init_m = put_data('frames')
subprocess.call(["ffmpeg", "-i", "frames/image-%04d.png", "-c:v", "libvpx", "temp.webm"])
print "done encoding"
subprocess.call(["ffmpeg", "-i", "temp.webm", "-vf", "scale=800:600", "outframes/image-%04d.png"])
print "done decoding"
final_m = get_data('outframes')

ctr = 0
for i, f in zip(init_m, final_m):
    if i == f:
        ctr += 1
print str(ctr/len(init_m) * 100) + "% of the qr codes were decoded"

#subprocess.call(["rm", "temp.webm"])
subprocess.call(["rm", "outframes/*"])



#frame = Image.open('frames/image-0422.png')
#f, m0 = qr_rgb_3_encode(frame)
#m1 = qr_rgb_3_decode(f)
#print(m0 == m1)